In [29]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 8: Resonant Collinearity ---</h2><p>You find yourselves on the <a href="/2016/day/25">roof</a> of a top-secret Easter Bunny installation.</p>
<p>While The Historians do their thing, you take a look at the familiar <em>huge antenna</em>. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand <span title="They could have imitated delicious chocolate, but the mediocre chocolate is WAY easier to imitate.">Imitation Mediocre</span> Chocolate as a Christmas gift! Unthinkable!</p>
<p>Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific <em>frequency</em> indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:</p>
<pre><code>............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
</code></pre>
<p>The signal only applies its nefarious effect at specific <em>antinodes</em> based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.</p>
<p>So, for these two antennas with frequency <code>a</code>, they create the two antinodes marked with <code>#</code>:</p>
<pre><code>..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
</code></pre>
<p>Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:</p>
<pre><code>..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
</code></pre>
<p>Antennas with different frequencies don't create antinodes; <code>A</code> and <code>a</code> count as different frequencies. However, antinodes <em>can</em> occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital <code>A</code> creates no antinodes but has a lowercase-<code>a</code>-frequency antinode at its location:</p>
<pre><code>..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
</code></pre>
<p>The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost <code>A</code>-frequency antenna:</p>
<pre><code>......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
</code></pre>
<p>Because the topmost <code>A</code>-frequency antenna overlaps with a <code>0</code>-frequency antinode, there are <code><em>14</em></code> total unique locations that contain an antinode within the bounds of the map.</p>
<p>Calculate the impact of the signal. <em>How many unique locations within the bounds of the map contain an antinode?</em></p>
</article>


In [30]:
from collections.abc import Iterator
from itertools import combinations


tests = [
    {
        "name": "Example wirh two",
        "s": """
            ..........
            ..........
            ..........
            ....a.....
            ..........
            .....a....
            ..........
            ..........
            ..........
            ..........
        """,
        "expected": 2,
    },
    {
        "name": "Example with three",
        "s": """
            ..........
            ..........
            ..........
            ....a.....
            ........a.
            .....a....
            ..........
            ..........
            ..........
            ..........
        """,
        "expected": 4,
    },
    {
        "name": "Example",
        "s": """
            ............
            ........0...
            .....0......
            .......0....
            ....0.......
            ......A.....
            ............
            ............
            ........A...
            .........A..
            ............
            ............
        """,
        "expected": 14,
    },
]
type Cell = tuple[int, int]


class AntennasMap(Str):
    def __init__(self, s: str) -> None:
        self.map = [list(l.strip()) for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.map), len(self.map[0])
        self.antennas = self._map_antennas()

    def antinodes(self, cell1: Cell, cell2: Cell) -> Iterator[Cell]:
        r1, c1 = cell1
        r2, c2 = cell2

        if 0 <= 2 * r1 - r2 < self.rows and 0 <= 2 * c1 - c2 < self.cols:
            yield 2 * r1 - r2, 2 * c1 - c2

        if 0 <= 2 * r2 - r1 < self.rows and 0 <= 2 * c2 - c1 < self.cols:
            yield 2 * r2 - r1, 2 * c2 - c1

    def unique_locations_with_an_antinode(self, debug=False) -> int:
        antinodes = set()
        for k, locs in self.antennas.items():
            for c1, c2 in combinations(locs, r=2):
                for r, c in self.antinodes(c1, c2):
                    antinodes.add((r, c))

        if debug:
            print(self.__str__(antinodes=antinodes))
        return len(antinodes)

    def _map_antennas(self) -> dict[str, list[Cell]]:
        antennas = defaultdict(list)
        for r, c in product(range(self.rows), range(self.cols)):
            if self.map[r][c] != ".":
                antennas[self.map[r][c]].append((r, c))
        return dict(antennas)

    def __str__(self, antinodes: set[Cell] = set()) -> str:
        return "\n".join(
            "".join(
                "#" if (cell == "." and (r, c) in antinodes) else cell
                for c, cell in enumerate(row)
            )
            for r, row in enumerate(self.map)
        )


@test(tests=tests[:])
def partI_test(s: str) -> int:
    am = AntennasMap(s)
    return am.unique_locations_with_an_antinode()


Test Example wirh two passed, for partI_test.
Test Example with three passed, for partI_test.
Test Example passed, for partI_test.
Success


In [31]:
with open("../input/day8.txt") as f:
    puzzle = f.read()

print(f"Part I: {AntennasMap(puzzle).unique_locations_with_an_antinode()}")

Part I: 348


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>348</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.</p>
<p>Whoops!</p>
<p>After updating your model, it turns out that an antinode occurs at <em>any grid position</em> exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).</p>
<p>So, these three <code>T</code>-frequency antennas now create many antinodes:</p>
<pre><code>T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
</code></pre>
<p>In fact, the three <code>T</code>-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to <code><em>9</em></code>.</p>
<p>The original example now has <code><em>34</em></code> antinodes, including the antinodes that appear on every antenna:</p>
<pre><code>##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
</code></pre>
<p>Calculate the impact of the signal using this updated model. <em>How many unique locations within the bounds of the map contain an antinode?</em></p>
</article>


In [32]:
from collections.abc import Iterator
from itertools import combinations
from string import ascii_letters


tests = [
    {
        "name": "Example T frequencies",
        "s": """
                T.........
                ...T......
                .T........
                ..........
                ..........
                ..........
                ..........
                ..........
                ..........
..........
        """,
        "expected": 9,
    },
    {
        "name": "Example",
        "s": """
            ............
            ........0...
            .....0......
            .......0....
            ....0.......
            ......A.....
            ............
            ............
            ........A...
            .........A..
            ............
            ............
        """,
        "expected": 34,
    },
]


class AntennasMapII(AntennasMap):
    def antinodes(self, cell1: Cell, cell2: Cell) -> Iterator[Cell]:
        r1, c1 = cell1
        r2, c2 = cell2

        dr = r2 - r1
        dc = c2 - c1

        rr, cc = cell2

        while 0 <= rr < self.rows and 0 <= cc < self.cols:
            yield rr, cc
            rr += dr
            cc += dc

        dr = r1 - r2
        dc = c1 - c2

        rr, cc = cell1

        while 0 <= rr < self.rows and 0 <= cc < self.cols:
            yield rr, cc
            rr += dr
            cc += dc


@test(tests=tests[:])
def partII_test(s: str) -> int:
    am = AntennasMapII(s)
    return am.unique_locations_with_an_antinode()


Test Example T frequencies passed, for partII_test.
Test Example passed, for partII_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [33]:
print(f"Part II: {AntennasMapII(puzzle).unique_locations_with_an_antinode()}")

Part II: 1221


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1221</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
